# Métricas

En este Notebook vamos analizar distintas métricas y algoritmos sobre un problema de clasificación desbalanceado. Ya hemos visto las principales métricas de regresión (MSE y RMSE) así que no nos detendremos en ellas aquí.

1. Análisis exploratorio
2. Métricas clasificación
3. Comparación clasificadores

Lo primero es cargar las librerías y funciones necesarias.

In [ ]:
from utils import plot_confusion_matrix, CM_BRIGHT

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Definimos una función para calcular y representar las métricas:

In [ ]:
def calcula_metricas(confmat):
    
    tn, fp, fn, tp = confmat.ravel()

    acc = (tp+tn)/(tn + fp + fn + tp)
    sen = tp/(tp+fn)
    esp = tn/(tn+fp)
    ppv = tp/(tp+fp)
    fsc = 2*(sen*ppv/(sen+ppv))

    print('ACC: ', acc)
    print('SEN: ', sen)
    print('ESP: ', esp)
    print('PPV: ', ppv)
    print('FSC: ', fsc)
    
    plt.bar(range(5),[acc,sen,esp,ppv,fsc])
    plt.xticks(range(5),['ACC','SEN','ESP','PPV','FSC'])
    plt.plot([-1, 6], [1, 1], color=(0.6, 0.6, 0.6), linestyle='--')
    plt.xlim((-0.5,4.5))
    plt.ylim((0,1.1))
    plt.title('Metricas')
    plt.show()

# 0. División train/test

Esta vez vamos a hacer las cosas bien hechas y dividiremos antes de hacer ningún tipo de análisis:

In [ ]:
from sklearn.model_selection import train_test_split

full_df = pd.read_csv('data/churn.csv', sep=',')
train, test = train_test_split(full_df, test_size=0.2, shuffle=True, stratify=full_df['churn'], random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./data/churn_train.csv', sep=',', index=False)
test.to_csv('./data/churn_test.csv', sep=',', index=False)

# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 

# 1. Análisis exploratorio

Vamos a trabajar con datos de fuga de una compañía telefónica. El objetivo es predecir si los clientes van a abandonar la compañía.

<div class = "alert alert-success">
EJERCICIO 8.1: Carga los datos *churn_train.csv* y realiza un primer análisis.
</div>

<div class = "alert alert-success">
EJERCICIO 8.2: Este problema está desbalanceado; calcula el ratio de desbalanceo.
</div>

In [ ]:
# ... código aquí: carga de datos
data = ...

In [ ]:
# ... código aquí: desbalanceo

##  1.1 Preprocesamiento de variables

Si escribimos *data.dtypes* nos indica el tipo de las variables de nuestro dataframe. Vemos que tenemos variables categóricas que tenemos que codificar:

In [ ]:
data.dtypes

<div class = "alert alert-success">
EJERCICIO 8.3: Elimine la variable *phone number* y codifique las variables categóricas con un Label Encoder.
</div>

In [ ]:
# ... código aquí: elimina phone number
data = data.drop(['phone number'], axis=1)
data.head().T

In [ ]:
# ... código aquí: codificación
from sklearn.preprocessing import LabelEncoder

le_state = LabelEncoder()

<div class = "alert alert-success">
EJERCICIO 8.4: Represente el histograma de las variable con distintos colores para cada clase.
</div>

In [ ]:
# ... código aquí: histogramas

## 1.2 Correlación entre variables

<div class = "alert alert-success">
EJERCICIO 8.5: Representa el mapa de correlación entre variables.
</div>

In [ ]:
import seaborn as sns

# ... código aquí: correlación

Podemos pintar las variables más correlacionadas (>0.95) con un scatter plot, para ver qué tipo de relación tienen:

In [ ]:
# Create correlation matrix
corr_matrix = data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.95
threshold = 0.95
pairs = np.where(upper>threshold)
fx = data.columns[pairs[0]]
fy =  data.columns[pairs[1]]

i=1
plt.figure(figsize=(22,4))
for f1,f2 in zip(fx,fy):
    
    plt.subplot(1,5,i)
    
    plt.scatter(data[f1],data[f2], c=data['churn'],cmap=CM_BRIGHT, alpha=0.25)
    plt.xlabel(f1)
    plt.ylabel(f2)
    plt.grid()
    plt.tight_layout()
    
    i+=1
    
plt.show()

Dada la correlación extrema y con el objetivo de eliminar variables poco informativas, podemos eliminar algunas columnas:

In [ ]:
columns_to_drop = ['total day minutes', 'total eve minutes', 'total night minutes', 'total intl minutes']
data = data.drop(columns_to_drop, axis=1)
data.head().T

Es buena idea agrupar todo el análisis y preprocesamiento en una única celda

In [ ]:
# RESUMO MI ANÁLISIS COMPLETO
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('data/churn_train.csv', sep=',')

# Elimino phone number
data = data.drop(['phone number'], axis=1)

# Codifico las variables categóricas
le_state = LabelEncoder()
le_ip = LabelEncoder()
le_vmp = LabelEncoder()
le_churn = LabelEncoder()

data['state'] = le_state.fit_transform(data['state'])
data['international plan'] = le_ip.fit_transform(data['international plan'])
data['voice mail plan'] = le_vmp.fit_transform(data['voice mail plan'])
data['churn'] = le_churn.fit_transform(data['churn'])

# Elimino columnas muy correlacionadas
columns_to_drop = ['total day minutes','total eve minutes','total night minutes','total intl minutes']
data = data.drop(columns_to_drop, axis=1)

print(data.shape)
data.head()

Porque así puedo aplicarlo muy fácilmente a test:

In [ ]:
data_test = pd.read_csv('data/churn_test.csv', sep=',')

# Elimino phone number
data_test = data_test.drop(['phone number'], axis=1)

# Codifico las variables categóricas
# con los mismos LabelEncoder de train, porque quiero conservar las clases

# data_test['state'].apply(x: x = 'Unknown' if x not in le_state.classes_)

data_test['state'] = le_state.transform(data_test['state'])
data_test['international plan'] = le_ip.transform(data_test['international plan'])
data_test['voice mail plan'] = le_vmp.transform(data_test['voice mail plan'])
data_test['churn'] = le_churn.transform(data_test['churn'])

# Elimino columnas muy correlacionadas
columns_to_drop = ['total day minutes','total eve minutes','total night minutes','total intl minutes']
data_test = data_test.drop(columns_to_drop, axis=1)

print(data_test.shape)
data_test.head()

# 2. Métricas en clasificación

Vamos a representar la matriz de confusión, y a partir de ella calcular distintas métricas. Para ello, comencemos un clasificador sencillo: regresión logística.

In [ ]:
from sklearn.preprocessing import StandardScaler

# preparamos los datos, un pelín distinto de otras veces
features = data.drop(['churn'], axis=1).columns

X_train = data[features].values
y_train = data['churn'].values

X_test = data_test[features].values
y_test = data_test['churn'].values

scaler = StandardScaler().fit(X_train)
Xs_train = scaler.transform(X_train)
Xs_test  = scaler.transform(X_test)

print('Datos train: ', Xs_train.shape)
print('Datos test:  ', Xs_test.shape)

print('Proporcion train:%0.3f'%np.mean(y_train))
print('Proporcion test: %0.3f'%np.mean(y_test))


## 2.1 Matriz de confusión y métricas

<div class = "alert alert-success">
EJERCICIO 8.6: Ajuste un algoritmo de regresión logística sobre el conjunto de entrenamiento con $C = 1$. Calcule la predicción para el conjunto de entrenamiento (*y_pred*).
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression

# ... código aquí
lr = ...
y_pred = lr.predict(Xs_train)

In [ ]:
from sklearn.metrics import confusion_matrix

confmat = confusion_matrix(y_train,y_pred)
plot_confusion_matrix(confmat)

# Podemos acceder a los valores de la matriz
tn, fp, fn, tp = confusion_matrix(y_train,y_pred).ravel()

In [ ]:
calcula_metricas(confmat)

<div class = "alert alert-success">
EJERCICIO 8.7: Calcule la predicción para el conjunto de test (*y_pred_test*).
</div>

In [ ]:
y_pred_test = ...

confmat = confusion_matrix(y_test,y_pred_test)
plot_confusion_matrix(confmat)
calcula_metricas(confmat)

Ahora representamos de nuevo los histogramas:

In [ ]:
y_prob = lr.predict_proba(Xs_test)[:,1]

idx_0 = (y_test==0)
idx_1 = (y_test==1)

plt.hist(y_prob[idx_0],density=1, alpha=0.75,label='y=0')
plt.hist(y_prob[idx_1],density=1, facecolor='red', alpha=0.75,label='y=1')
plt.legend()

plt.show()

Por último, vamos a representar la curva ROC.

In [ ]:
from sklearn.metrics import roc_curve, auc

ejex, ejey, _ = roc_curve(y_test, y_prob)
roc_auc = auc(ejex, ejey)

plt.figure()
plt.plot(ejex, ejey, color='darkorange',lw=2, label='AUC = %0.2f' % roc_auc)

plt.plot([0, 1], [0, 1], color=(0.6, 0.6, 0.6), linestyle='--')
plt.plot([0, 0, 1],[0, 1, 1],lw=2, linestyle=':',color='black',label='Clasificador perfecto')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])

plt.xlabel('FPR (1-ESP)')
plt.ylabel('SEN')
plt.legend(loc="lower right")
plt.show()

# 3. Comparación clasificadores

Vamos a comparar los siguientes clasificadores: 

* Regresión logística
* Árboles de decisión
* Random Forest

## 3.1 Regresión logística

Hemos visto que este algoritmo está sesgado hacia la clase mayoritoria. Para compensar esta situación, podemos asignar pesos distintos a los errores cometidos en cada una de las clases, a través del parámetro [*class_weight*](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

Además, podemos trabajar con distintas [métricas](http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) a la hora de optimizar los parámetros libres. Para conjuntos desbalancedados es adecuada 'f1': F1-score, compromiso entre SEN y PPV.


In [ ]:
from sklearn.model_selection import GridSearchCV

vectorC = np.logspace(-3,3,21)
param_grid = {'C': vectorC }

grid = GridSearchCV(LogisticRegression(random_state=0, class_weight='balanced'),
                    scoring='accuracy', 
                    param_grid=param_grid, 
                    cv = 10)

grid.fit(Xs_train, y_train)

print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

scores = grid.cv_results_['mean_test_score']
std_scores = grid.cv_results_['std_test_score']
plt.errorbar(np.log10(vectorC),scores,yerr=std_scores, fmt='o-',ecolor='g')
plt.xlabel('log(C)',fontsize=16)
plt.ylabel('10-Fold MSE')
plt.grid()
plt.show()

<div class = "alert alert-success">
EJERCICIO 8.8: El código de arriba optimiza balanceado y con accuracy. Compare el resultado con respecto a entrenar tres combinaciones: sin balancear + accuracy; sin balancear + F1; balanceado + F1.
</div>

In [ ]:
# ...código aquí: not balanced + ACC
grid = ...
Copt = grid.best_params_['C']

lr = ...
y_pred = lr.predict(Xs_test)

confmat_test  = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(confmat_test)
calcula_metricas(confmat_test)

In [ ]:
# ...código aquí: not balanced + F1
grid = ...
Copt = grid.best_params_['C']

lr = ...
y_pred = lr.predict(Xs_test)

confmat_test  = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(confmat_test)
calcula_metricas(confmat_test)

In [ ]:
# ...código aquí: balanced + F1
grid = ...
Copt = grid.best_params_['C']

lr = ...
y_pred = lr.predict(Xs_test)

confmat_test  = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(confmat_test)
calcula_metricas(confmat_test)

Vamos a representar histogramas para esta última:

In [ ]:
y_prob = lr.predict_proba(Xs_test)[:,1]

idx_0 = (y_test==0)
idx_1 = (y_test==1)

plt.hist(y_prob[idx_0],density=1, alpha=0.75,label='y=0')
plt.hist(y_prob[idx_1],density=1, facecolor='red', alpha=0.75,label='y=1')
plt.legend()

plt.show()

Y la curva ROC:

In [ ]:
from sklearn.metrics import roc_curve, auc

ejex, ejey, _ = roc_curve(y_test, y_prob)
roc_auc = auc(ejex, ejey)

plt.figure()
plt.plot(ejex, ejey, color='darkorange',lw=2, label='AUC = %0.2f' % roc_auc)

plt.plot([0, 1], [0, 1], color=(0.6, 0.6, 0.6), linestyle='--')
plt.plot([0, 0, 1],[0, 1, 1],lw=2, linestyle=':',color='black',label='Clasificador perfecto')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])

plt.xlabel('FPR (1-ESP)')
plt.ylabel('SEN')
plt.legend(loc="lower right")
plt.show()

## 3.2 Árboles de decisión

Entrenamos ahora un árbol de decisión. Otra ventaja adicional de los árboles es que por su construcción hace frente al desbalanceo de las clases.

<div class = "alert alert-success">
EJERCICIO 8.9: Entrena un árbol de decisión y calcula las métricas obtenidas en el conjunto de test.
</div>

In [ ]:
from sklearn.tree import DecisionTreeClassifier

maxDepth = range(1,15)
param_grid = {'max_depth': maxDepth }

# ... código aquí
grid = ...
grid.fit(Xs_train, y_train)
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

scores = np.array(grid.cv_results_['mean_test_score'])
plt.plot(maxDepth,scores,'-o')
plt.xlabel('max_depth',fontsize=16)
plt.ylabel('10-Fold MSE')
plt.show()

In [ ]:
maxDepthOptimo = grid.best_params_['max_depth']
# ... código aquí
treeModel = ...

print("Train: ",treeModel.score(Xs_train,y_train))
# fun fact: me equivoqué al copiar y dejé X_test. Haced la prueba, a ver qué pasa.
print("Test: ",treeModel.score(Xs_test,y_test)) 

y_pred = treeModel.predict(Xs_test)

In [ ]:
# ... código aquí
confmat_test = ...
plot_confusion_matrix(confmat_test)
calcula_metricas(confmat_test)

Como es un árbol individual, podemos representarlo:

In [ ]:
from sklearn import tree

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(treeModel, feature_names=list(features), filled=True)

## 3.3 Random Forest

Comprobemos prestaciones para un algoritmo de Random Forest.

<div class = "alert alert-success">
EJERCICIO 8.10: Entrena un algoritmo de Random Forest y calcula las métricas obtenidas en el conjunto de test.
</div>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# grid search
maxDepth   = range(1,15)
param_grid = {'max_depth': maxDepth}

# ... código aquí
grid = ...
grid.fit(X_train, y_train)

print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

scores = np.array(grid.cv_results_['mean_test_score'])
plt.plot(maxDepth,scores,'-o')
plt.xlabel('max_depth')
plt.ylabel('10-fold ACC')

plt.show()

In [ ]:
maxDepthOptimo = grid.best_params_['max_depth']
# ... código aquí
rf = ...

print("Train: ",rf.score(Xs_train,y_train))
print("Test: ",rf.score(Xs_test,y_test)) 

y_pred = rf.predict(Xs_test)

In [ ]:
# ... código aquí
confmat_test = ...
plot_confusion_matrix(confmat_test)
calcula_metricas(confmat_test)

Con esto podríais evaluar cualquier algoritmo de sklearn, no sólo los que hemos visto en clase. Simplemente encontrad el (o los) parámetro que regula la complejidad, y ajustadlo con validación cruzada.